### DỰ ĐOÁN THỜI GIAN GIAO HÀNG

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import statistics
from geopy.distance import geodesic
# CT geodesic trong thư viện Geopy sử dụng các tọa độ địa lý của 2 điểm
# (như Restaurant_latitude, Restaurant_longitude, Delivery_location_latitude, Delivery_location_longitude) 
# để tính toán khoảng cách giữa chúng trên bề mặt của Trái Đất. 
# Cho phép tính khoảng cách giữa các vị trí dựa trên tọa độ địa lý của chúng.

from sklearn.model_selection import train_test_split,cross_val_score, GridSearchCV
from sklearn.preprocessing import LabelEncoder,StandardScaler 
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.metrics import mean_squared_error, r2_score,mean_absolute_error
import warnings
warnings.filterwarnings('ignore')
pd.set_option("display.max_columns",None)
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

In [ ]:
df_train = pd.read_csv('C:/Users/WELCOME/Desktop/DATTNT/Đồ án TTNT/Dataset/train.csv')
df_train.head()

In [ ]:
df_train.columns

In [ ]:
print("Train Dataset :", df_train.shape)

In [ ]:
df_train.info()

# Kiểm tra các số liệu thống kê có trong bảng với dữ liệu định tính 

In [ ]:
#Kiểm tra giá trị thống kê cho các trường có kiểu dữ liệu số
df_train.describe().T

In [ ]:
#Kiểm tra giá trị thống kê cho các trường có kiểu dữ liệu số khác với kiểu dữ liệu số, freq: số lần xuất hiện
df_train.describe(exclude=np.number).T


### Sau khi quan sát, ta đã rút ra được:

1. Có nhiều giá trị null ở trong cột "Time_Orderd".
2. Cần phải chỉnh lại loại DL cho cột "Weatherconditions" và "Time_taken(min)"
3. Cả hai dữ liệu định lượng (đo or đếm đc) và định tính ( 0 đo đc, chỉ để phân loại) đều có trong dataset.
4. ID & Delivery_person_ID là 2 cột không cần thiết trong việc xây dựng mô hình.

### Khám phá từng cột một

In [ ]:
#Explore each column
for column in df_train.columns:
    print(column)
    print(df_train[column].value_counts()) #đếm slg gtri duy nhất trong mỗi cột
    print("------------------------------------")

- ID: Có 45593 giá trị duy nhất trong cột ID.
- Delivery_person_ID: Có một số ID của shipper xuất hiện nhiều lần.
- Weatherconditions: Có một số điều kiện thời tiết xuất hiện nhiều lần như "Sunny" hoặc "Cloudy".
- Type_of_order: Loại đơn hàng "Snack" xuất hiện nhiều nhất.
- Type_of_vehicle: Xe máy là phương tiện phổ biến nhất
- Festival: Khách thường mua hàng vào dịp không phải ngày lễ
- City: Có nhiều đơn hàng được thực hiện ở các thành phố khác nhau.

# Data Cleaning

### Chỉnh sửa tên cột

In [ ]:
def update_column_name(df):
    #Đổi tên cột Weatherconditions
    df.rename(columns={'Weatherconditions': 'Weather_conditions'},inplace=True)
    
update_column_name(df_train)
print(df_train.columns)

### Biến đổi các giá trị trong cột thành những giá trị dễ đọc 

In [ ]:

def extract_column_value(df):
    #Chỉnh sửa cột Time và chuyển dữ liệu thành loại int
    df['Time_taken(min)'] = df['Time_taken(min)'].apply(lambda x: int(x.split(' ')[1].strip()))
    # chia thành 2 phần = khoảng trắng, sau đó chỉ lấy phần thứ 2 (số phút), rồi chuyển số phút -> int
    
    #Chỉnh sửa cột Weather conditions
    df['Weather_conditions'] = df['Weather_conditions'].apply(lambda x: x.split(' ')[1].strip())
    #Thêm mã thành phố từ Delivery person ID
    df['City_code']=df['Delivery_person_ID'].str.split("RES", expand=True)[0]
    
extract_column_value(df_train)
df_train[['Time_taken(min)','Weather_conditions','City_code']].head()

### Xóa những cột không dùng trong việc xây dựng Model

In [ ]:

def drop_columns(df):
    df.drop(['ID','Delivery_person_ID'],axis=1,inplace=True)
    
print("Before No. of columns: ",df_train.shape[1])
drop_columns(df_train)
print("After No. of columns: ",df_train.shape[1])

### Kiểm tra các giá trị trùng lặp

In [ ]:

if (len(df_train[df_train.duplicated()])>0):
    print("There are Duplicate values present")
else:
    print("There is no duplicate value present")

### Chỉnh sửa kiểu dữ liệu phù hợp 

In [ ]:

def update_datatype(df):
    df['Delivery_person_Age'] = df['Delivery_person_Age'].astype('float64')
    df['Delivery_person_Ratings'] = df['Delivery_person_Ratings'].astype('float64')
    df['multiple_deliveries'] = df['multiple_deliveries'].astype('float64')
    df['Order_Date']=pd.to_datetime(df['Order_Date'],format="%d-%m-%Y")
    
update_datatype(df_train)

### Biến đổi string "NaN" thành np.nan
(np.nan là gtri đặc biệt để biểu thị gtri missing value)

In [ ]:
def convert_nan(df):
    df.replace('NaN', float(np.nan), regex=True,inplace=True)

convert_nan(df_train)

### Kiểm tra các giá trị null

In [ ]:

df_train.isnull().sum().sort_values(ascending=False)

In [ ]:
#Handle null values
def handle_null_values(df):
    df['Delivery_person_Age'].fillna(np.random.choice(df['Delivery_person_Age']), inplace=True)
    df['Weather_conditions'].fillna(np.random.choice(df['Weather_conditions']), inplace=True)
    df['City'].fillna(df['City'].mode()[0], inplace=True) #mode: điền = gtri xuất hiện nhiều nhất
    df['Festival'].fillna(df['Festival'].mode()[0], inplace=True)
    df['multiple_deliveries'].fillna(df['multiple_deliveries'].mode()[0], inplace=True)
    df['Road_traffic_density'].fillna(df['Road_traffic_density'].mode()[0], inplace=True)
    df['Delivery_person_Ratings'].fillna(df['Delivery_person_Ratings'].median(), inplace=True)
    
handle_null_values(df_train)
df_train.isnull().sum()

### Handling outliers 

In [ ]:
upper_limit = df_train['Delivery_person_Age'].mean() + 3 * df_train['Delivery_person_Age'].std()
upper_limit

In [ ]:
lower_limit = df_train['Delivery_person_Age'].mean() - 3*df_train['Delivery_person_Age'].std()
lower_limit

In [ ]:
df_train = df_train[(df_train['Delivery_person_Age'] < upper_limit) & (df_train['Delivery_person_Age'] > lower_limit)]
df_train

In [ ]:
df_train = df_train[~(df_train["Restaurant_latitude"] == 0)]\
    .reset_index(drop=True).copy()
df_train

In [ ]:
df_train = df_train[~(df_train["Delivery_location_latitude"] == 0)]\
    .reset_index(drop=True).copy()
df_train

In [ ]:
df_train = df_train[~(df_train["Restaurant_latitude"] < 7.00)]\
    .reset_index(drop=True).copy()
df_train

### Kiểm tra xem một lần nữa dataset đã sạch chưa 

In [ ]:
for column in df_train.columns:
    print(column)
    print(df_train[column].value_counts())
    print("------------------------------------")

In [ ]:
obj=df_train.select_dtypes(include=['object'])
num=df_train.select_dtypes(include=['int64','float64'])
print(obj.columns) #DL dạng chuỗi
print(num.columns) #DL dạng số

In [ ]:
obj=['Weatherconditions', 'Road_traffic_density', 'Type_of_order',
       'Type_of_vehicle', 'multiple_deliveries', 'Festival', 'City']
num=['Delivery_person_Age', 'Delivery_person_Ratings', 'Vehicle_condition']

df_train[num]

# Data Visualization

In [ ]:
df_train['Delivery_person_Age'] = df_train['Delivery_person_Age'].astype('float')

In [ ]:
sns.displot(df_train['Delivery_person_Age'])
#biểu đồ phân phối

### Delivery_person_Ratings

In [ ]:
df_train['Delivery_person_Ratings']=df_train['Delivery_person_Ratings'].astype('float64')

In [ ]:
sns.displot(df_train['Delivery_person_Ratings'])

### Type_of_Order

In [ ]:
plt.figure(dpi=100)
sns.countplot(data=df_train,x='Type_of_order')


In [ ]:
plt.figure(dpi=100)
sns.countplot(data=df_train,x='Type_of_vehicle')

In [ ]:
sns.countplot(x=df_train.City,hue=df_train.Type_of_order)
#đô thị, tp đô thị, ngoại ô

In [ ]:
sns.countplot(x=df_train.City,hue=df_train.Road_traffic_density)
#sự tắc nghẽn giao thông


In [ ]:
sns.countplot(x=df_train.Type_of_order,hue=df_train.Festival) #Festival doesn't affect the food ordering frequency

In [ ]:
sns.countplot(x=df_train.Vehicle_condition,hue=df_train.City) #Vehicle condition is overall good in all 3 cities
#It has taken more time to deliver food in semi-urban cities, likely due to traffic or road conditions.

In [ ]:
plt.figure(dpi=100)
sns.kdeplot(data=df_train,x='Time_taken(min)', hue='Type_of_vehicle', fill=True)
# most of the delivery is done through motorcylce and also have much higher mean delivery time compared to others

### Heatmap

In [ ]:
# plt.figure(figsize=(12,12))
# sns.heatmap(df_train.corr(),annot=True,linewidths=0.6,fmt=".2f",cmap="coolwarm")

# Correlation HeatMap
plt.figure(figsize=(15,9))
sns.heatmap(df_train.select_dtypes(include=['float64', 'int64']).corr(), annot=True, linewidth=0.1, cmap="Blues")

plt.show()


# Feature Engineering

In [ ]:
def extract_date_features(data):
    data["day"] = data.Order_Date.dt.day
    data["month"] = data.Order_Date.dt.month
    data["quarter"] = data.Order_Date.dt.quarter
    data["year"] = data.Order_Date.dt.year
    data['day_of_week'] = data.Order_Date.dt.day_of_week.astype(int)
    data["is_month_start"] = data.Order_Date.dt.is_month_start.astype(int)
    data["is_month_end"] = data.Order_Date.dt.is_month_end.astype(int)
    data["is_quarter_start"] = data.Order_Date.dt.is_quarter_start.astype(int)
    data["is_quarter_end"] = data.Order_Date.dt.is_quarter_end.astype(int)
    data["is_year_start"] = data.Order_Date.dt.is_year_start.astype(int)
    data["is_year_end"] = data.Order_Date.dt.is_year_end.astype(int)
    #Ngày thứ 5 và thứ 6 trong tuần là thứ sáu và thứ bảy
    data['is_weekend'] = np.where(data['day_of_week'].isin([5,6]),1,0)

extract_date_features(df_train)
df_train.head()

In [ ]:
# #Tính khoảng cách thời gian
# def calculate_time_diff(df):
#     # Tính khoảng cách thời gian từ khi đặt hàng đến khi shipper lấy đơn hàng 
#     df['Time_Orderd'] = pd.to_timedelta(df['Time_Orderd'])
#     df['Time_Order_picked'] = pd.to_timedelta(df['Time_Order_picked'])
    
#     df['Time_Order_picked_formatted'] = df['Order_Date'] + np.where(df['Time_Order_picked'] < df['Time_Orderd'], pd.DateOffset(days=1), pd.DateOffset(days=0)) + df['Time_Order_picked']
#     df['Time_Ordered_formatted'] = df['Order_Date'] + df['Time_Orderd']
    
#     df['order_prepare_time'] = (df['Time_Order_picked_formatted'] - df['Time_Ordered_formatted']).dt.total_seconds() / 60
    
#     # Xử lý các giá trị null bằng cách điền vào giá trị trung vị
#     df['order_prepare_time'].fillna(df['order_prepare_time'].median(), inplace=True)
    
#     # Bỏ tất cả các cột liên quan đến ngày và giờ
#     df.drop(['Time_Orderd', 'Time_Order_picked', 'Time_Ordered_formatted', 'Time_Order_picked_formatted', 'Order_Date'], axis=1, inplace=True)


# calculate_time_diff(df_train)
# df_train.head()





def calculate_time_diff(df):
    # Chuyển đổi các cột thời gian thành định dạng datetime
    df['Order_Date'] = pd.to_datetime(df['Order_Date'])
    df['Time_Orderd'] = pd.to_timedelta(df['Time_Orderd'])
    df['Time_Order_picked'] = pd.to_timedelta(df['Time_Order_picked'])
    
    # Tính toán thời gian đặt hàng và thời gian lấy hàng
    df['Time_Order_picked_formatted'] = df['Order_Date'] + np.where(df['Time_Order_picked'] < df['Time_Orderd'], pd.DateOffset(days=1), pd.DateOffset(days=0)) + df['Time_Order_picked']
    df['Time_Ordered_formatted'] = df['Order_Date'] + df['Time_Orderd']
    
    # Chuyển đổi các cột thành kiểu datetime nếu chúng không phải là kiểu datetime
    df['Time_Order_picked_formatted'] = pd.to_datetime(df['Time_Order_picked_formatted'])
    df['Time_Ordered_formatted'] = pd.to_datetime(df['Time_Ordered_formatted'])
    
    # Tính toán thời gian chuẩn bị đơn hàng
    df['order_prepare_time'] = (df['Time_Order_picked_formatted'] - df['Time_Ordered_formatted']).dt.total_seconds() / 60
    
    # Xử lý giá trị null bằng cách điền vào giá trị trung vị
    df['order_prepare_time'].fillna(df['order_prepare_time'].median(), inplace=True)
    
    # Bỏ tất cả các cột liên quan đến ngày và giờ
    df.drop(['Time_Orderd', 'Time_Order_picked', 'Time_Ordered_formatted', 'Time_Order_picked_formatted', 'Order_Date'], axis=1, inplace=True)

calculate_time_diff(df_train)
df_train.head()







In [ ]:
#Tính toán khoảng cách giữa địa điểm nhà hàng và địa điểm giao hàng
def calculate_distance(df):
    df['distance']=np.zeros(len(df))
    restaurant_coordinates=df[['Restaurant_latitude','Restaurant_longitude']].to_numpy()
    delivery_location_coordinates=df[['Delivery_location_latitude','Delivery_location_longitude']].to_numpy()
    df['distance'] = np.array([geodesic(restaurant, delivery) for restaurant, delivery in zip(restaurant_coordinates, delivery_location_coordinates)])
    df['distance']= df['distance'].astype("str").str.extract('(\d+)').astype("int64")
    
calculate_distance(df_train)
df_train.head()

In [ ]:
#nháp thử

# Correlation HeatMap
plt.figure(figsize=(15,9))
sns.heatmap(df_train.select_dtypes(include=['float64', 'int64']).corr(), annot=True, linewidth=0.1, cmap="Blues")

plt.show()


# Data Preprocessing

## Label Encoding

In [ ]:
def label_encoding(df):
    categorical_columns = df.select_dtypes(include='object').columns
    label_encoder = LabelEncoder()
    df[categorical_columns] = df[categorical_columns].apply(lambda col: label_encoder.fit_transform(col))

label_encoding(df_train)
df_train.head(10)


## Train test split

In [ ]:
# train = df_train.loc[df_train.index < 33198]
# test = df_train.loc[df_train.index >= 33198]

#Split features & label
X = df_train.drop('Time_taken(min)', axis=1)  # Features
y = df_train['Time_taken(min)']  # dự đoán

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

## Standardization

In [ ]:
# Tạo một đối tượng có bộ chia tỷ lệ tiêu chuẩn
scaler = StandardScaler()

# Phù hợp với bộ chia tỷ lệ trên dữ liệu đào tạo
scaler.fit(X_train)

# Chuẩn hóa bộ dữ liệu train
X_train = scaler.transform(X_train)

# Chuẩn hóa trên bộ dữ liệu test
X_test = scaler.transform(X_test)



# Model Building

### Các bước thực hiện
**1. Sử dụng xác thực chéo và điều chỉnh siêu tham số để xác định mô hình hồi quy tối ưu.**

**2. Xây dựng mô hình dự đoán bằng cách sử dụng mô hình tốt nhất đã được xác định.**

**3. Đánh giá hiệu suất của mô hình trên dữ liệu thử nghiệm để đánh giá độ chính xác và độ tin cậy của nó.**

### Cross Validation

In [ ]:
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb

# Tìm mô hình tốt nhất
models = [
    LinearRegression(),
    DecisionTreeRegressor(),
    RandomForestRegressor(),
    xgb.XGBRegressor(),
]

param_grid = [
    {},  
    {'max_depth': [3, 5, 7]},
    {'n_estimators': [100, 200, 300]},
    {'n_estimators': [20, 25, 30], 'max_depth': [5, 7, 9]},
]

for i, model in enumerate(models):
    grid_search = GridSearchCV(model, param_grid[i], cv=5, scoring='r2')
    grid_search.fit(X_train, y_train)

    print(f"{model.__class__.__name__}:")
    print("Best parameters:", grid_search.best_params_)
    print("Best R2 score:", grid_search.best_score_)
    print()


# Future Prediction

### Model Building

In [ ]:
# Create a XGB regressor model
model = xgb.XGBRegressor(n_estimators=20,max_depth=9)

# Fit the model on the training data
model.fit(X_train, y_train,
         eval_set = [(X_train, y_train), (X_test, y_test)], verbose = True)

**rmse giá trị càng thấp thì độ chính xác của mô hình càng cao**

### Model Evaluation

In [ ]:
# Make predictions on the test data
y_pred = model.predict(X_test)
y_pred
# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print("Mean Absolute Error (MAE):", round(mae,2))
print("Mean Squared Error (MSE):", round(mse,2))
print("Root Mean Squared Error (RMSE):", round(rmse,2))
print("R-squared (R2) Score:", round(r2,2))

In [ ]:
df_train.columns

In [ ]:
test['prediction_time'] = model.predict(X_test)
def drop_columns(df):
    df.drop(['Delivery_person_Age', 'Delivery_person_Ratings', 'Restaurant_latitude',
       'Restaurant_longitude', 'Delivery_location_latitude',
       'Delivery_location_longitude', 'Weather_conditions',
       'Road_traffic_density', 'Vehicle_condition', 'Type_of_order',
       'Type_of_vehicle', 'multiple_deliveries', 'Festival', 'City', 'City_code', 'day', 'month', 'quarter', 'year',
       'day_of_week', 'is_month_start', 'is_month_end', 'is_quarter_start',
       'is_quarter_end', 'is_year_start', 'is_year_end', 'is_weekend',
       'order_prepare_time', 'distance'],axis=1,inplace=True)
drop_columns(test)

In [ ]:
test.tail(100)

In [ ]:
test.head(100)

# Conclusion

**In conclusion, the food delivery prediction model was developed using XGBoost, achieving an impressive R2 score of 0.82. Moving forward, potential enhancements include identifying the best features, conducting additional feature engineering, and exploring other optimization techniques to further improve the model's performance and accuracy. These steps will contribute to fine-tuning the model and unlocking its full potential in predicting food delivery timings accurately.**


Trong kết luận, mô hình dự đoán giao hàng đồ ăn đã được phát triển bằng cách sử dụng XGBoost, đạt được điểm R2 ấn tượng là 0.82. Đối với các bước tiếp theo, các cải tiến tiềm năng bao gồm xác định các đặc trưng tốt nhất, tiến hành thêm công việc tạo đặc trưng, và khám phá các kỹ thuật tối ưu hóa khác để cải thiện hiệu suất và độ chính xác của mô hình. Những bước này sẽ đóng góp vào việc điều chỉnh mô hình và mở khóa toàn bộ tiềm năng của nó trong việc dự đoán thời gian giao hàng đồ ăn một cách chính xác.